In [2]:
from sampo.generator import SimpleSynthetic, SyntheticGraphType
from random import Random

rand = Random(123)
ss = SimpleSynthetic(rand)

fixed_wg = ss.work_graph(mode=SyntheticGraphType.GENERAL, cluster_counts=10, bottom_border=100, top_border=100)

fixed_wg.vertex_count

106

In [3]:
from sampo.schemas.stochastic_graph import ProbabilisticFollowingStochasticGraphScheme

# construct the stochastic graph scheme
graph_scheme = ProbabilisticFollowingStochasticGraphScheme(rand=rand, wg=fixed_wg)
defect_graphs = []
for i, node in enumerate(fixed_wg.nodes):
    defect = ss.graph_nodes(top_border=20)
    defect_graphs.append(defect)
    
    defect_prob = rand.random() * 0.5
    graph_scheme.add_part(node=node.id, nodes=defect, prob=defect_prob)

perfect_wg = graph_scheme.prepare_graph().to_work_graph()
print(perfect_wg.vertex_count)

Can not find native module; switching to default
349


In [4]:
graph_scheme.prepare_graph().to_work_graph().vertex_count

330

In [5]:
from sampo.schemas import WorkerProductivityMode, IntervalGaussian
from sampo.schemas.time_estimator import DefaultWorkEstimator, WorkTimeEstimator


def construct_work_estimator(i: int, productivity: WorkerProductivityMode) -> WorkTimeEstimator:
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(productivity)
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker)
    return work_estimator

work_estimator = construct_work_estimator(5, WorkerProductivityMode.Static)

In [6]:
from sampo.scheduler.heft.prioritization import prioritization

static_prioritization = prioritization(fixed_wg, work_estimator)
static_prioritization

[ac7c2e69-7f2e-eb7e-0352-05d79b683076,
 8ff41e8d-8944-897c-610e-4ad509c47055,
 0d496d4b-da22-594f-573e-8d77550ae889,
 f692ff1b-e245-f7f3-29f2-a2fbf83d58c3,
 dea61187-b7bc-5c51-1072-75716b461015,
 8f9b34c9-5653-d4bd-2293-202c28dfa298,
 ab33e90c-5f42-937f-24dc-25a5add65955,
 29ed479c-3ec3-068c-b3a1-f1315573ae2d,
 d3234519-1410-7b06-be7e-41743c269046,
 1a1f141b-72c5-3633-bb66-ecb350c1eb44,
 6e3e43c2-e941-2995-4ab3-8105056fb885,
 6ea37f12-62e1-0745-96dc-0a90cd620098,
 73211a9a-597b-6ecf-0546-d9e6ab1f5a09,
 737e7bea-ab38-ba7e-b605-d0dffafe0f4e,
 4a8bbf55-ea73-6384-c5bf-60c6a7eb2407,
 7ba35975-cd2e-e163-858c-2e6e57ed22d0,
 8e7bea51-83c4-a23e-844f-cbcfdf13b884,
 17c57804-570f-89e9-91f7-aa69e4b80fdf,
 e497cd2f-65f2-ef76-0336-5a0850b51605,
 3e44ede9-819c-f976-ee08-6a5045917fb4,
 6f33f708-8343-b946-a56a-62c3c4ef7978,
 0e0eaf52-527d-eda5-9afe-de82d1de950d,
 3993539e-5e76-d1e0-b9f0-82f18ba31e52,
 7ea9de52-fde6-2703-49a6-dbeda38873b0,
 96020f97-0d35-ea15-616e-068ff1b8788c,
 b4abf550-49e8-ea0d-f206-

In [7]:
stochastic_wg = graph_scheme.prepare_graph().to_work_graph()
stochastic_wg.vertex_count

319

In [8]:
from sampo.scheduler.heft.prioritization import prioritization_nodes
from sampo.schemas import WorkGraph, GraphNode


def stochastic_prioritization_function(wg: WorkGraph, work_estimator: WorkTimeEstimator) -> list[GraphNode]:
    # wg is stochastic-generated graph
    # here we can use only information from static graph
    # let's use static prioritization

    # TODO Still no followers saved... find problem please
    order = []
    for original_static_node in static_prioritization:
        static_node = stochastic_wg[original_static_node.id]
        order.append(static_node)
        # haha, get all defects...
        children = static_node.get_following_nodes()
        children_prioritization = prioritization_nodes(children, work_estimator)
        order.extend(children_prioritization)
    return order

In [9]:
from sampo.generator.environment import get_contractor_by_wg
from sampo.pipeline import SchedulingPipeline
from sampo.scheduler import HEFTScheduler

scheduler = HEFTScheduler(prioritization_f=stochastic_prioritization_function, work_estimator=work_estimator)
contractors = [get_contractor_by_wg(stochastic_wg)]

project = SchedulingPipeline.create() \
    .wg(stochastic_wg) \
    .contractors(contractors) \
    .work_estimator(work_estimator) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

1697

In [10]:
project.schedule.full_schedule_df

,idx,task_id,task_name,task_name_mapped,contractor,cost,volume,measurement,start,finish,duration,workers,scheduled_work_object
0,0,6364d5e0-d65a-2a5e-70e1-ad82c4710bb7,minimal road,minimal road,cf11c5a3-a039-477a-9fe1-dae3401060f4,1770,6.009262,km,0,1,1,"{'driver': 53, 'manager': 17, 'handyman': 107}",ScheduledWork[work_unit=6364d5e0-d65a-2a5e-70e...
1,1,ac7c2e69-7f2e-eb7e-0352-05d79b683076,start of project,start of project,cf11c5a3-a039-477a-9fe1-dae3401060f4,0,0.000000,unit,0,0,0,{},ScheduledWork[work_unit=ac7c2e69-7f2e-eb7e-035...
2,2,ce4aedc2-608f-38c8-1d15-98a5601977ae,start of project,start of project,cf11c5a3-a039-477a-9fe1-dae3401060f4,0,0.000000,unit,0,0,0,{},ScheduledWork[work_unit=ce4aedc2-608f-38c8-1d1...
3,3,e5182800-e282-110b-eda7-ffc2d14f568f,temporary road,temporary road,cf11c5a3-a039-477a-9fe1-dae3401060f4,1770,6.009262,km,1,2,1,"{'driver': 53, 'manager': 17, 'handyman': 107}",ScheduledWork[work_unit=e5182800-e282-110b-eda...
4,4,2e27cdd5-ac91-0a39-f166-f41b5bd840c6,final road,final road,cf11c5a3-a039-477a-9fe1-dae3401060f4,3540,6.009262,km,2,4,2,"{'driver': 53, 'manager': 17, 'handyman': 107}",ScheduledWork[work_unit=2e27cdd5-ac91-0a39-f16...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,311,0895efeb-7fa0-c012-587c-0ed0ff078e91,final road,final road,cf11c5a3-a039-477a-9fe1-dae3401060f4,2290,9.236569,km,1694,1695,1,"{'driver': 67, 'manager': 27, 'handyman': 135}",ScheduledWork[work_unit=0895efeb-7fa0-c012-587...
312,312,f8b1a70c-9636-e016-d5b4-8ea4f6c7c21b,cluster handing,cluster handing,cf11c5a3-a039-477a-9fe1-dae3401060f4,80,0.760000,unit,1695,1696,1,"{'driver': 2, 'manager': 2, 'engineer': 4}",ScheduledWork[work_unit=f8b1a70c-9636-e016-d5b...
313,313,fed4829c-2b03-621a-48fb-d18640dc5438,cluster handing,cluster handing,cf11c5a3-a039-477a-9fe1-dae3401060f4,80,0.960000,unit,1696,1697,1,"{'driver': 2, 'manager': 2, 'engineer': 4}",ScheduledWork[work_unit=fed4829c-2b03-621a-48f...
314,314,44faa95b-0bef-5bcf-32bf-e8fa2a2e562e,finish of project,finish of project,cf11c5a3-a039-477a-9fe1-dae3401060f4,0,0.000000,unit,1697,1697,0,{},ScheduledWork[work_unit=44faa95b-0bef-5bcf-32b...


In [ ]:
from sampo.scheduler.heft.prioritization import stochastic_prioritization

scheduler = HEFTScheduler(stochastic_prioritization_f=stochastic_prioritization, work_estimator=work_estimator)
contractors = [get_contractor_by_wg(stochastic_wg)]

project = SchedulingPipeline.create() \
    .wg(stochastic_wg) \
    .contractors(contractors) \
    .work_estimator(work_estimator) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

In [ ]:
# [node for lst in self._node2followers.values() for lst2, _ in lst for node in lst2 if node.work_unit.name == 'finish of project']
# [(g1, g2) for g1 in defect_graphs for g2 in defect_graphs if len(set(g1).intersection(set(g2))) > 0]
# sorted([node for lst in self._node2followers.values() for lst2, _ in lst for node in lst2], key=lambda x: x.id)
# sorted([(node, prob) for lst in self._node2followers.values() for lst2, prob in lst for node in lst2], key=lambda x: x[0].id)